## 태스크 1: 모델 훈련

기계 학습(ML) 모델 생성 프로세스는 데이터 처리부터 시작합니다. 데이터 처리가 완료되면 모델을 훈련시킬 ML 알고리즘을 선택합니다. 모델 훈련의 목표는 향후 데이터로 예측하는 데 사용할 수 있는 모델을 생성하는 것입니다. 처리한 데이터에는 대상이 포함되어야 하지만 향후 데이터에는 대상이 포함되지 않습니다(레이블이 지정되지 않음). 이 알고리즘은 입력 데이터 속성을 대상에 매핑하는 패턴을 훈련 데이터에서 찾습니다. 그런 후 알고리즘은 이러한 패턴을 캡처하는 ML 모델을 출력합니다. 모델이 있는 경우 대상 값이 포함되지 않는 새 데이터에 대해 예측을 수행할 수 있습니다.

예를 들어, 이메일이 스팸인지 여부를 예측하도록 ML 모델을 훈련시키려면 대상을 알 수 있는 이메일을 포함하는 훈련 데이터(이 경우 이메일이 스팸인지 여부를 알려 주는 레이블)를 모델에 제공합니다. 이 데이터를 사용하여 알고리즘은 이메일이 스팸인지 여부를 예측하는 모델을 생성합니다. 이 모델을 사용해서 향후 이메일 레이블을 예측할 수 있습니다.

이 태스크에서는 누군가의 소득이 50,000 USD 미만인지 여부를 예측합니다. 모델은 누군가의 소득이 50,000 USD 미만인지를 최대한 정확하게 예측할 수 있도록 훈련을 통해 자체적으로 최적화됩니다. 모델 훈련을 위해서는 훈련하는 데 사용하려는 알고리즘 종류를 비롯한 몇 가지 구성이 필요합니다. 이 태스크에서는 XGBoost(eXtreme 그래디언트 부스팅) 알고리즘을 사용합니다. 모델을 훈련시킬 때 하이퍼파라미터도 구성해야 합니다. 하이퍼파라미터는 훈련 작업 프로세스를 제어하는 파라미터입니다. 훈련 작업의 다양한 단계를 변경하도록 조정될 수 있습니다. 적절한 하이퍼파라미터 세트를 선택하는 일은 모델 성능 및 정확도 측면에서 중요합니다. 모델을 훈련시킨 후에는 모델을 평가하고 모델 아티팩트를 확인합니다.

### 태스크 1.1: 환경 설정

모델 훈련을 시작하기 전에 필요한 종속 항목을 모두 설치하십시오.

In [ ]:
#Install matplotlib and restart kernel
%pip install matplotlib
%pip uninstall bokeh -y
%pip install bokeh==2.4.2
%reset -f

# Install dependencies
import boto3
import io
import json
import math
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import sagemaker
import sys
import time
import zipfile

from sagemaker.debugger import Rule, rule_configs
from IPython.display import FileLink, FileLinks
from sagemaker import image_uris
from IPython.display import display
from IPython.display import Image
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.inputs import TrainingInput
from sagemaker.session import Session
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.xgboost.estimator import XGBoost
from time import gmtime, strftime
from pathlib import Path

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')

참고: 출력에 “ERROR: pip's dependency resolver does not currently take into account all the packages that are installed(ERROR: pip 종속성 확인자는 현재 설치된 모든 패키지를 고려하지는 않습니다.)’ 오류가 표시되는 경우 패키지 종속성 오류를 무시하고 다음 단계를 진행하면 됩니다.

다음으로는 데이터세트를 가져옵니다. 이 연습에서는 누군가가 Amazon SageMaker Data Wrangler에서 Amazon Simple Storage Service(Amazon S3) 버킷으로 데이터 파일을 내보냈다고 가정합니다. 이 데이터세트를 훈련(70%), 검증(20%), 테스트(10%) 데이터세트로 분할합니다. 훈련 및 검증 데이터세트는 훈련 중에 사용됩니다. 테스트 데이터세트는 배포 후의 모델 평가에 사용됩니다.

Amazon SageMaker를 사용하여 모델을 훈련시키려면 데이터세트가 libSVM 또는 CSV 형식이어야 합니다. 이 실습에서는 훈련에 CSV 형식을 사용합니다. 

이전 실습에서 생성한 데이터세트 파일을 보려면 아래의 단계를 따르십시오.

1. 왼쪽 메뉴 모음에서 버킷 아이콘을 선택합니다.

1. 버킷 목록에서 이름에 **labdatabucket**이 포함된 Amazon S3 버킷을 선택합니다.

.csv 파일을 열면 SageMaker Studio에서 새 탭이 열립니다. 여기서 설명하는 지침에 따라 작업을 진행하려면 다음 옵션 중 하나를 사용하십시오.
- **옵션 1**: 탭을 나란히 표시합니다. 주 SageMaker Studio 창에서 분할 화면 보기를 생성하려면 **lab_2.ipynb** 탭을 옆쪽으로 끌거나 **lab_2.ipynb** 탭을 마우스 오른쪽 버튼으로 클릭하여 선택한 후 **New View for Notebook**을 선택합니다. 그러면 .csv 파일을 살펴볼 때 지침을 표시할 수 있습니다.
- **옵션 2**: SageMaker Studio 탭을 서로 전환하면서 지침에 따라 작업을 진행합니다. .csv 파일 탐색을 완료한 후 **lab_2.ipynb** 탭을 선택하여 노트북으로 돌아옵니다.

1. **scripts** 폴더, **data** 폴더, **train** 폴더 및 **adult_data_processed_train.csv** 파일을 차례로 선택(두 번 클릭)하여 해당 내용을 확인합니다.

1. 왼쪽 창의 <i aria-hidden="true" class="fas fa-folder" style="color:white"></i> **/ ... /data/train/** 브레드크럼 링크에서 **data**를 선택합니다.

1. **validation** 폴더, **adult_data_processed_validation.csv** 파일을 차례로 선택(두 번 클릭)하여 해당 내용을 확인합니다.

데이터세트 파일을 확인했습니다. 이제 훈련 작업이 해당 입력으로 사용하는 훈련 및 검증 경로를 구성합니다.

In [ ]:
# Import the datasets
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)
prefix = 'scripts/data'
output_path = 's3://{}/{}/output'.format(bucket, prefix)

# Configure the training paths
train_path = f"s3://{bucket}/{prefix}/train/adult_data_processed_train.csv"
validation_path = f"s3://{bucket}/{prefix}/validation/adult_data_processed_validation.csv"

# Set up the TrainingInput objects
train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')

# Print the training and validation paths
print(f'Training path: {train_path}')
print(f'Validation path: {validation_path}')

# Set the container, name, and tags
create_date = strftime("%m%d%H%M")
container = image_uris.retrieve(framework='xgboost',region=boto3.Session().region_name,version='1.5-1')
run_name = 'lab-2-run-{}'.format(create_date)

### 태스크 1.2: 추정기 객체 구성

추정기는 SageMaker 훈련을 위한 고급 인터페이스입니다. AWS Identity and Access Management(AWS IAM) 역할, 컴퓨팅 인스턴스 수 및 유형, Amazon S3 출력 경로와 같은 필수 파라미터를 제공하여 추정기 객체를 생성합니다. 이 실습에서는 SageMaker 일반 추정기에 대한 XGBoost 기본 제공 알고리즘을 사용합니다. XGBoost는 그래디언트 부스트 트리 알고리즘에서 유명하고 효율적인 오픈 소스 구현입니다. 그래디언트 부스팅은 더욱 단순하고 약한 모형 세트의 추정치의 앙상블을 결합하여 대상 변수를 정확하게 예측하려 시도하는 지도 학습 알고리즘입니다. XGBoost 알고리즘은 다양한 데이터 유형, 관계, 배포 및 미세 조정할 수 있는 다양한 하이퍼파라미터를 잘 처리합니다. 회귀, 분류(바이너리 및 멀티클래스) 및 순위 문제에 대해 XGBoost를 사용할 수 있습니다. 이 경우 XGBoost를 사용하여 분류 문제(누군가의 소득이 50,000 USD 미만인지 여부)를 해결합니다.

이 실습에서는 *sagemaker.estimator.Estimator* 클래스를 사용하여 XGBoost 추정기를 생성합니다. 다음 예제 코드에서 XGBoost 추정기 이름은 *xgb_model*입니다. SageMaker 추정기를 생성하려면 다음 파라미터를 지정합니다.

- **image_uri**: 훈련 컨테이너 이미지 URI. 이 예제에서 SageMaker XGBoost 훈련 컨테이너 URI는 *image_uris.retrieve*를 사용하여 지정합니다.
- **role**: SageMaker가 사용자 대신 태스크(예: 훈련 결과 읽기, Amazon S3에서 모델 아티팩트 호출, Amazon S3에 훈련 결과 쓰기)를 수행하는 데 사용하는 IAM 역할. 
- **instance_count and instance_type**: 모델 훈련에 사용할 Amazon EC2 ML 컴퓨팅 인스턴스의 유형 및 수. 이 실습의 경우 CPU 4개, 16GB 메모리, Amazon Elastic Block Store(Amazon EBS) 스토리지, 높은 네트워크 성능을 갖는 단일 ml.m5.xlarge 인스턴스를 사용합니다.
- **output_path**: SageMaker가 모델 아티팩트 및 훈련 결과를 저장하는 S3 버킷의 경로.
- **sagemaker_session**: SageMaker API 작업과 훈련 작업이 사용하는 다른 AWS 서비스와의 상호 작용을 관리하는 세션 객체.
- **rules**: Amazon SageMaker Debugger 기본 제공 규칙 목록. 이 예제에서 create_xgboost_report() 규칙은 훈련 프로세스 및 결과에 대한 인사이트를 제공하는 XGBoost 보고서를 생성합니다.

In [ ]:
xgb_model = sagemaker.estimator.Estimator(
    image_uri = container,
    role = role, 
    instance_count = 1, 
    instance_type ='ml.m5.xlarge',
    output_path = output_path,
    sagemaker_session = sagemaker_session,
    rules=[
        Rule.sagemaker(rule_configs.create_xgboost_report())
    ]
)

### 태스크 1.3: 하이퍼파라미터 구성

하이퍼파라미터는 모델 구조, 기능 및 성능을 직접 제어합니다. 하이퍼파라미터 튜닝을 통해 데이터 과학자는 최적의 결과를 위해 모델 성능을 조정합니다. 이 프로세스는 기계 학습의 필수적 부분으로, 적절한 하이퍼파라미터 값을 선택하는 것이 성공에 중요한 요인입니다.

추정기의 *set_hyperparameters* 메서드를 호출하여 XGBoost 알고리즘에 대한 하이퍼파라미터를 설정할 수 있습니다.

XGBoost 하이퍼파라미터에 관한 자세한 내용은 [XGBoost 하이퍼파라미터](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html)를 참조하십시오.

In [ ]:
xgb_model.set_hyperparameters(
    max_depth = 5,
    eta = 0.2,
    gamma = 4,
    min_child_weight = 6,
    subsample = 0.7,
    verbosity = 0,
    objective = 'binary:logistic',
    num_round = 800
)

### 태스크 1.4: SageMaker 훈련 작업 실행

지금까지 추정기 객체 및 하이퍼파라미터를 구성했으므로 모델을 훈련시킬 준비가 되었습니다. fit() 메서드는 훈련 스크립트를 시작합니다. 모델 훈련을 시작하려면 훈련 및 검증 데이터세트로 추정기의 fit() 메서드를 호출합니다. `wait=True`를 설정하는 경우 fit() 메서드는 진행 로그를 표시하고 훈련이 완료될 때까지 기다립니다.

In [ ]:
xgb_model.fit(
    {
        "train": train_input,
        "validation": validation_input
    },
    wait=True
)

<i aria-hidden="true" class="fas fa-clipboard-check" style="color:#18ab4b"></i> **Expected output:** 추정기 및 하이퍼파라미터 구성이 올바르고 훈련 작업이 올바르게 시작된 경우 다음 출력이 표시됩니다.

```plain
************************
**** 출력 예시 ****
************************

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-08-09-20-09-56-628
2023-08-09 20:09:56 Starting - Starting the training job...
2023-08-09 20:10:19 Starting - Preparing the instances for trainingCreateXgboostReport: InProgress
......
2023-08-09 20:11:21 Downloading - Downloading input data...
2023-08-09 20:11:55 Training - Downloading the training image...
2023-08-09 20:12:20 Training - Training image download completed. Training in progress....
2023-08-09 20:12:56 Uploading - Uploading generated training model...
2023-08-09 20:13:20 Completed - Training job completed
..Training seconds: 107
Billable seconds: 107
```

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i> **Note:** 훈련을 실행하려면 약 3~4분이 소요됩니다.

### 태스크 1.5: 모델 평가

훈련 작업이 완료된 후에는 SageMaker Debugger에서 생성된 XGBoost 훈련 보고서를 다운로드할 수 있습니다. XGBoost 훈련 보고서는 반복과 관련된 손실 함수, 특성 중요도, 혼동 행렬, 정확도 곡선 및 기타 훈련의 통계 결과와 같은 훈련 진행 및 결과에 대한 인사이트를 제공합니다. 

SageMaker XGBoost 훈련 작업의 경우 디버거 `CreateXgboostReport` 규칙을 사용하여 훈련 진행 및 결과에 대한 포괄적인 훈련 보고서를 수신합니다.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i> **Note:** 보고서를 완료하는 데 5~10분 정도 소요될 수 있습니다. 다음 코드 블록에서는 최종 보고서 유무를 확인합니다. 보고서가 아직 없으면 30초 후 시스템이 출력 폴더의 새 파일을 로컬 경로에 복사하고 보고서가 준비될 때까지 보고서 유무를 다시 확인합니다.

In [ ]:
%%capture
the_file = Path("CreateXgboostReport/xgboost_report.ipynb")
notexists = True
while notexists:
    if the_file.is_file():
        notexists = False
    else:
        time.sleep(30)
        rule_output_path = xgb_model.output_path + "/" + xgb_model.latest_training_job.job_name + "/rule-output"
        ! aws s3 ls {rule_output_path} --recursive
        ! aws s3 cp {rule_output_path} ./ --recursive

다음 셀 출력에 포함된 링크를 클릭하면 SageMaker Studio에서 새 탭이 열립니다. 여기서 설명하는 지침에 따라 작업을 진행하려면 다음 옵션 중 하나를 사용하십시오.
- **옵션 1**: 탭을 나란히 표시합니다. 주 SageMaker Studio 창에서 분할 화면 보기를 생성하려면 **lab_2.ipynb** 탭을 옆쪽으로 끌거나 **lab_2.ipynb** 탭을 마우스 오른쪽 버튼으로 클릭하여 선택한 후 **New View for Notebook**을 선택합니다. 그러면 XGBoost 보고서를 살펴볼 때 지침을 표시할 수 있습니다.
- **옵션 2**: SageMaker Studio 탭을 서로 전환하면서 지침에 따라 작업을 진행합니다. XGBoost 보고서 탐색을 완료한 후 **lab_2.ipynb** 탭을 선택하여 노트북으로 돌아옵니다.

In [ ]:
display("Click link below to view the XGBoost Training notebook", FileLink("CreateXgboostReport/xgboost_report.ipynb"))

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i> **참고: ** 이 코드를 실행하면 출력으로 **'Click link below to view the XGBoost Training notebook' <span style="ssb_sm_blue">CreateXgboostReport/xgboost_report.ipynb</span>**가 표시됩니다.

새 탭에서 노트북을 열려면 링크를 선택합니다. 노트북이 열리면 **Set up notebook environment** 창에서 다음을 구성합니다.

- **Image**에서 **Data Science 3.0**을 선택합니다.
- **Kernel**(커널)에서 **Python 3**를 선택합니다.
- **Select**(선택)를 선택합니다.

**xgboost_report.ipynb** 탭 상단에서 <i aria-hidden="true" class="fas fa-forward"></i> **Restart Kernel and Run All Cells**(커널 다시 시작 및 모든 셀 실행) 버튼을 선택합니다. **Restart Kernel?**(커널을 다시 시작하시겠습니까?) 메시지가 표시되면 **Restart**(다시 시작)를 선택합니다.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i> **Note:** 모든 셀을 실행하려면 2~3분 정도 걸립니다.

모든 셀이 실행 완료되면 아래쪽으로 스크롤하여 **Confusion Matrix**를 선택합니다. 혼동 행렬은 관찰 결과의 예측 클래스와 실제 클래스를 비교하여 각 클래스에 대해 올바른 예측 및 잘못된 예측 수를 표로 나타냅니다. 다이어그램으로 이동하면 **참 긍정(TP)**, **참 부정(TN)**, **거짓 긍정(FP)** 및 **거짓 부정(FN)** 값이 표시됩니다.

- **참 긍정:** 실제 분류가 긍정이고 예측한 분류가 긍정(1,1)이면 긍정 샘플이 분류자에서 올바르게 분류된 것이므로 **참 긍정(TP)** 결과라고 합니다. 
- **거짓 부정:** 실제 분류가 긍정이고 예측한 분류가 부정(1,0)이면 긍정 샘플이 분류자에서 부정으로 잘못 분류된 것이므로 **거짓 부정(FN)** 결과라고 합니다. 
- **거짓 긍정:** 실제 분류가 부정이고 예측한 분류가 긍정(0,1)이면 부정 샘플이 분류자에서 긍정으로 잘못 분류된 것이므로 **거짓 긍정(TP)** 결과라고 합니다. 
- **참 부정:** 실제 분류가 부정이고 예측한 분류가 부정(0,0)이면 부정 샘플이 분류자에서 올바르게 분류된 것이므로 **참 부정(TN)** 결과라고 합니다.

다음으로 아래로 스크롤하여 **혼동 행렬의 평가**로 이동한 후 **분류 보고서**를 좀 더 자세히 확인하여 각 클래스의 정밀도, 재현율 및 F1 점수의 요약을 이해합니다.

- **정밀도**: 긍정으로 예측되는 모든 예제 중에서 긍정으로 예측된 실제 긍정값의 비율을 측정합니다. 범위는 0~1이고 값이 클수록 정확도가 더 좋은 것입니다. 정밀도는 모델이 관련성이 있다고 나타낸 데이터 포인트 중에서 실제로 관련성이 있는 비율을 나타냅니다. 정밀도는 특히 FP 비용이 높을 때 고려하기에 좋은 측정값입니다.
- **재현율/민감도/참 긍정 비율(TPR)**: 긍정으로 예측된 실제 긍정값의 비율을 측정합니다. 이 범위도 0~1이며 값이 클수록 예측 정확도가 더 높은 것입니다. 이것을 재현율/민감도라고도 합니다. 이 측정값은 데이터세트의 모든 관련 인스턴스를 찾는 능력을 나타냅니다.
- **F1 점수**: 정밀도 및 재현율의 조화 평균에 해당하는 대상 지표를 나타냅니다. F1은 FP 및 FN을 둘 다 고려하여 정밀도와 재현율에 동일한 가중치를 부여합니다.

자격 있는 시민에게 정부 지원 서비스를 홍보할 수 있도록 소득이 50,000 USD 미만인 경우를 예측하려고 합니다. 이 경우에 F1 점수는 FP(소득이 50,000 USD보다 높지만 50,000 USD 미만으로 레이블이 지정된 사람) 및 FN(소득이 50,000 USD보다 높은 것으로 레이블이 지정된 50,000 USD 미만인 사람)을 모두 고려하므로 사용하기에 적절한 측정값입니다. 정밀도 및 재현율이 모두 높으며 F1 점수가 두 측정값을 모두 고려하는지 확인하려고 합니다. 다음 실습에서는 하이퍼파라미터를 튜닝하면서 더 높은 F1 점수를 얻을 수 있는지 확인하여 모델을 최적화합니다.

이 모델의 **Precision**, **Recall**, **F1 Score** 및 **Overall Accuracy**는 무엇입니까?

<i aria-hidden="true" class="far fa-comment" style="color:#008296"></i> **Consider:** 노트북에 포함된 다른 그래프를 잠시 검토해 보십시오. 어떤 종류의 정보가 표시됩니까? 모델을 훈련시킬 때 도움이 될 만한 정보로는 어떤 것이 있습니까?

### 태스크 1.6: 모델 아티팩트 확인

SageMaker는 S3 버킷에 모델 아티팩트를 저장합니다. 모델 아티팩트의 위치를 찾으려면 다음 단계를 따르십시오.

1. 왼쪽 메뉴 모음에서 버킷 아이콘을 선택합니다.

1. 버킷 목록에서 이름에 **labdatabucket**이 포함된 Amazon S3 버킷을 엽니다.

1. **scripts/data/output/ sagemaker-xgboost-.../output** 하위 폴더로 이동합니다. 

하위 폴더에 모델 아티팩트 **model.tar.gz**가 표시됩니다. 이것은 fit() 메서드를 호출하여 SageMaker Estimator로 생성한 모델입니다.

model.tar.gz 파일을 비롯한 모델 아티팩트를 확인했습니다. 

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가 태스크 2 부분을 계속 진행합니다.